In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("paper", font_scale=2) 

In [5]:
%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

In [6]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [7]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

### **Задание 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - np.clip).**

In [8]:
#Вариант из вебинара
def calc_logloss2(y, y_pred):
    y_pred = np.clip(y_pred, 0.00001, 0.99999)
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [9]:
y_true = 1
y_pred = 0
calc_logloss2(y_true, y_pred)

11.512925464970229

### Задание 2. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [10]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # подходит или нет репетитор
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [11]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [12]:
np.random.seed(42)
w = np.random.randn(X_st.shape[1])
n = X.shape[0]
alpha = 18
diff = 1
count = 0
while diff > 1e-7:
    z = np.dot(X_st, w)
    y_pred = sigmoid(z)  
    err1 = calc_logloss2(y, y_pred)
    w -= alpha * (1/n * np.dot(X_st.T, (y_pred - y)))
    z = np.dot(X_st, w)
    y_pred = sigmoid(z)
    err2 = calc_logloss2(y, y_pred)   
    diff = abs(err1 - err2)
    count += 1
  # alpha = alpha / count
print(f'Итоговые коэффициенты = {w}, mse = {err2}, количество итераций = {count}, {diff}')

Итоговые коэффициенты = [-228.00918407  -16.03932456  -67.59062895  174.38533755], mse = 0.006239361834215295, количество итераций = 52059, 9.999995137684181e-08


In [13]:
#при альфа = 0.001 mse = 0.2838526592243291, количество итераций = 600462
#при альфа = 0.01 mse = 0.1457 достиг за 547339 итераций
#при альфа = 0.1 mse = 0.0810808668854147, количество итераций = 284635
#при альфа = 0.2 mse = 0.06359735902033659, количество итераций = 269196
#при альфа = 0.5 mse = 0.04024848375942085, количество итераций = 257825
#при альфа = 1 mse = 0.027783377826263455, количество итераций = 216856
#при альфа = 10 mse = 0.008411540443540758, количество итераций = 78507
#при альфа = 11 mse = 0.008012913073426099, количество итераций = 73919
#при альфа = 18 mse = 0.006239361834215295, количество итераций = 52059
#при альфа = 19 mse = 5.756467732512389, количество итераций = 2
#При альфа = 20 mse = 5.756467732512389, количество итераций = 2



In [14]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w)
        y_pred = sigmoid(z)
        err = calc_logloss2(y, y_pred)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
#         if i % (iterations / 10000) == 0:
#             print(i, w, err)
    return w, err

In [15]:
w, err = eval_LR_model(X_st, y, iterations=52059, alpha=18)

In [16]:
print(w, err)

[-228.00918407  -16.03932456  -67.59062895  174.38533755] 0.0062394618341666715


### Задание 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).

In [17]:
def calc_pred_proba(X, w):
    return 1 / (1 + np.exp(-np.dot(X, w)))

In [18]:
proba = calc_pred_proba(X_st, w)
print(proba)

[3.07066558e-02 2.47379302e-14 1.00000000e+00 2.79943451e-08
 9.98394053e-01 2.02044274e-13 1.00000000e+00 2.96559804e-04
 9.71181612e-01 1.00000000e+00]


In [19]:
sigmoid(np.dot(X_st, w))

array([3.07066558e-02, 2.47379302e-14, 1.00000000e+00, 2.79943451e-08,
       9.98394053e-01, 2.02044274e-13, 1.00000000e+00, 2.96559804e-04,
       9.71181612e-01, 1.00000000e+00])

### Задание 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred).


In [20]:
def calc_pred(X, w):
    
    y_predicted = []  
    
    A = calc_pred_proba(X, w)
    
#     За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(len(A)):
        if (A[i] > 0.5): 
            y_predicted.append(1)
        elif (A[i] <= 0.5):
            y_predicted.append(0)
    
    return y_predicted

In [21]:
y_pred = calc_pred(X_st, w)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]

In [22]:
y

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### Задание 5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [23]:
def calc_matrix_error(y, y_pred):
    
    matrix_error = np.array([[0,0],[0,0]])
    
    for i in range(len(y)):
        if y[i] == 1:
            if y[i] == y_pred[i]:
                matrix_error[0][0] += 1 # true positive
            else:
                matrix_error[1][0] += 1 # false negative
        if y[i] == 0:
            if y[i] == y_pred[i]:
                matrix_error[1][1] += 1 # true negative
            else:
                matrix_error[0][1] += 1 #false positive

    return matrix_error
                

In [24]:
ma = calc_matrix_error(y, y_pred)
ma

array([[5, 0],
       [0, 5]])

In [25]:
def calc_accuracy(matrix_error):
    return (matrix_error[0][0] + matrix_error[1][1]) / np.sum(matrix_error)

In [26]:
acc = calc_accuracy(ma)
acc

1.0

In [27]:
def calc_precision(matrix_error):
    return matrix_error[0][0] / (matrix_error[0][0] + matrix_error[0][1])

In [28]:
prec = calc_precision(ma)
prec

1.0

In [29]:
def calc_recall(matrix_error):
    return matrix_error[0][0] / (matrix_error[0][0] + matrix_error[1][0])

In [30]:
recall = calc_recall(ma)
recall

1.0

In [31]:
def calc_f1_score(matrix_error):
    return (2 * calc_precision(matrix_error) * calc_recall(matrix_error)) / (calc_precision(matrix_error) + calc_recall(matrix_error))

In [32]:
r1_score = calc_f1_score(ma)
print(r1_score)

1.0


### Задание 6. Могла ли модель переобучиться? Почему?

Не должна. В нашей моделе нет коэффициентов регуляризации, к-ые могли бы слишком изменить веса, нет полиномов, к-ые также могли значительно изменить веса ("подогнаться" под данные). С другой стороны, если данные имеют нелинейный характер взаимодействия (те же выбросы) модель вынужена будет построить модель неадекватно предсказывая по новым данным.

### Задание 7. 1. *Создайте функции eval_LR_model_l1 и eval_LR_model_l2 с применением L1 и L2 регуляризации соответственно.

In [33]:
def eval_LR_model_l1(X, y, iterations, C = 0.1, alpha=1e-4):
    
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w)
        y_pred = sigmoid(z)
        err = calc_logloss2(y, y_pred)
        new_w = w - alpha * (1/n * np.dot(X.T, (y_pred - y)) + 1/C * np.sign(w))
        w = new_w
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w, err

In [34]:
w, err = eval_LR_model_l1(X_st, y, iterations=60000, C = 1, alpha=0.01)

6000 [-0.00617369 -0.00591328 -0.00747247  0.00029739] 0.6918591027394162
12000 [-0.00616203 -0.00576435 -0.00841025 -0.00412981] 0.6925146417516286
18000 [-0.00614771 -0.0058041  -0.00940753 -0.00857947] 0.6931914397616195
24000 [-0.00613452 -0.00563585  0.00967208  0.00701885] 0.6938801504893164
30000 [-0.00615783 -0.00582367  0.00863877  0.00247381] 0.6917756850386132
36000 [-0.00617498 -0.00572201  0.0077271  -0.0019697 ] 0.6924249013397505
42000 [-0.00617353 -0.00570559  0.00673883 -0.00644889] 0.6931008415571951
48000 [-0.00609954 -0.00548683  0.00578264  0.00923982] 0.6937736276131661
54000 [-0.00614324 -0.00547113  0.0048561   0.00472276] 0.6944906256582986
60000 [-0.00618174 -0.00580451  0.00376686  0.00012311] 0.692351258401595


In [35]:
print(w, err)

[-0.00618174 -0.00580451  0.00376686  0.00012311] 0.692351258401595


In [36]:
def eval_LR_model_l2(X, y, iterations, C = 0.1, alpha=1e-4):
    
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w)
        y_pred = sigmoid(z)
        err = calc_logloss2(y, y_pred)
        new_w = w - alpha * (1/n * np.dot(X.T, (y_pred - y)) + 2 * С * w)
        w = new_w
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w, err